In [255]:
import requests, re, datetime
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup as bs

In [320]:
# Retrieve HTML data
url = "http://ke.customer.decent.cz/a022/mon/"
filename = "meu_data.xlsx"

soup = bs(requests.get(url).content, 'html5lib')
tables = soup.find_all('table')

In [321]:
# Get and format timestamp from first table.
raw_datetime = tables[2].get_text()
date = re.search(r'(\d+).(\d+).(\d+)', raw_datetime)
time = re.search(r'(\d\d)\s(\d\d)', raw_datetime)

timestamp = datetime.datetime(
    int(date.group(3)),
    int(date.group(2)),
    int(date.group(1)), 
    int(time.group(1)),
    int(time.group(2)))

In [325]:
# Get data and import it into a Pandas DataFrame, format it.
raw_data = tables[4]
rows = raw_data.find_all('tr')
rows = [x.find_all('td') for x in rows]
data = [[col.get_text() for col in row] for row in rows]

headers = ['activity', 'queue', 'x','active_booths', 'wait_time','x']

table = pd.DataFrame( data[1:], columns=headers).set_index('activity')[:11]
table['queue'] = table['queue'].str.extract("(\d+)", expand=False)
table['wait_time'] = table['wait_time'].str.extract("(\d+)", expand=False).fillna(0)
table['active_booths'] = table['active_booths'].str.extract("(\d+)", expand=False)
table = table.drop('x', axis=1)

# table_queue = pd.DataFrame(table['queue'])
# table_queue.columns = [timestamp]

# table_active_booths = pd.DataFrame(table['active_booths'])
# table_active_booths.transpose()

table_wait_time = pd.DataFrame(table['wait_time'])

# Create file if doesn't exist
if Path.is_file(Path(filename)):
    table_base = pd.read_excel(filename)
    print(table_base)
    
# data_all = {}

# for column in table:
#     temp = []
# #     print(table[column])
#     temp = pd.DataFrame(table[column])
#     temp.columns = [timestamp]
#     data_all[column] = temp

# table_all = pd.Panel(data_all)
# table_all.to_excel(filename, encoding='utf-8')

XLRDError: No sheet named <'r'>